In [ ]:
import cv2

def calculate_motion_mask(previous_frame, current_frame):
    # Ensure both frames are in BGR color space
    bgr_previous = cv2.cvtColor(previous_frame, cv2.COLOR_RGB2BGR)
    bgr_current = cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR)
   
    # Convert frames to grayscale
    gray_prev = cv2.cvtColor(bgr_previous, cv2.COLOR_BGR2GRAY)
    gray_curr = cv2.cvtColor(bgr_current, cv2.COLOR_BGR2GRAY)
   
    # Calculate frame difference
    frame_diff = cv2.absdiff(gray_prev, gray_curr)
   
    # Apply threshold to get motion mask
    _, motion_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
   
    return motion_mask

def detect_events(motion_mask, frame):
    contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"Event Detected", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            timestamp = int(cv2.getTickCount() / cv2.getTickFrequency())
            cv2.imwrite(f"event_{timestamp}.jpg", frame)

video_path = "iva.mp4"
cap = cv2.VideoCapture(video_path)

previous_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
   
    if previous_frame is None:
        previous_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        continue
   
    # Calculate motion mask
    motion_mask = calculate_motion_mask(previous_frame, cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
   
    # Update previous frame
    previous_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
    # Detect events
    detect_events(motion_mask, frame)
   
    # Display the resulting frame
    cv2.imshow('Motion Detection', frame)
   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
from fer import FER
import numpy as np

def load_image_set(image_paths):
    """Load a set of images."""
    return [cv2.imread(path) for path in image_paths]

def detect_faces(image):
    """Detect faces using Haar cascade face detection."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    return faces

def classify_emotion(face_image):
    """Classify emotion using a pre-trained FER model."""
    emotion_detector = FER(mtcnn=True)
    result = emotion_detector.detect_emotions(face_image)
    
    if result:
        # Return the emotion with the highest confidence
        dominant_emotion = max(result[0]['emotions'], key=result[0]['emotions'].get)
        return dominant_emotion
    else:
        return "neutral"

def categorize_image_sentiment(individual_sentiments):
    """Categorize overall sentiment of the image."""
    happy_count = sum(1 for s in individual_sentiments if s == 'happy')
    sad_count = sum(1 for s in individual_sentiments if s == 'sad')
    
    if happy_count > sad_count:
        return 'happy'
    elif sad_count > happy_count:
        return 'sad'
    else:
        return 'neutral'

def analyze_crowd(image_paths):
    """Analyze crowd sentiment."""
    images = load_image_set(image_paths)
    
    for i, image in enumerate(images):
        print(f"Processing image {i+1}...")
        
        faces = detect_faces(image)
        individual_sentiments = []
        
        for (x, y, w, h) in faces:
            face_image = image[y:y+h, x:x+w]
            emotion = classify_emotion(face_image)
            individual_sentiments.append(emotion)
            
            # Display sentiments and facial features
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        overall_sentiment = categorize_image_sentiment(individual_sentiments)
        print(f"Overall Image {i+1} sentiment: {overall_sentiment}")
        
        cv2.imshow(f"Image {i+1}", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Example usage
image_paths = ['img1.jpeg', 'img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg', 'img7.jpg']
analyze_crowd(image_paths)

Processing image 1...
Overall Image 1 sentiment: neutral
Processing image 2...
Overall Image 2 sentiment: happy
Processing image 3...
Overall Image 3 sentiment: happy
Processing image 4...
Overall Image 4 sentiment: sad
Processing image 5...
Overall Image 5 sentiment: happy
Processing image 6...
Overall Image 6 sentiment: sad
Processing image 7...
Overall Image 7 sentiment: sad


In [25]:
import cv2
import numpy as np

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load OpenCV's deep learning face detector
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

# Define MODEL_MEAN_VALUES
MODEL_MEAN_VALUES = (78.42633776026858, 102.15891835115144, 117.08628437568004)

# Load gender detection model
gender_prototxt = 'gender_deploy.prototxt'
gender_model = 'gender_net.caffemodel'
gender_net = cv2.dnn.readNet(gender_prototxt, gender_model)

# Define gender classes
gender_classes = ['Male', 'Female']

def detect_faces(image):
    # Detect faces using Haar cascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces_haar = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(30, 30))

    # Detect faces using deep learning
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104.0, 177.0, 123.0])
    net.setInput(blob)
    faces_dnn = net.forward()

    # Combine results and apply strict criteria
    combined_results = []
    for (x, y, w, h) in faces_haar:
        if w < image.shape[1] // 2 and h < image.shape[0] // 2:  # Check if face is within half the image size
            combined_results.append((x, y, w, h, 0.8))  # Default confidence for Haar cascade
    for i in range(faces_dnn.shape[2]):
        confidence = faces_dnn[0, 0, i, 2]
        if confidence > 0.7:  # Stricter confidence threshold
            x = int(faces_dnn[0, 0, i, 3] * image.shape[1])
            y = int(faces_dnn[0, 0, i, 4] * image.shape[0])
            w = int(faces_dnn[0, 0, i, 5] * image.shape[1])
            h = int(faces_dnn[0, 0, i, 6] * image.shape[0])
            if w < image.shape[1] // 2 and h < image.shape[0] // 2:  # Check if face is within half the image size
                combined_results.append((x, y, w, h, confidence))

    # Apply non-maximum suppression twice
    def nms_boxes(detections, threshold):
        if len(detections) > 0:
            detections = sorted(detections, key=lambda x: x[4], reverse=True)
            suppressed = []
            for i in range(len(detections)):
                if detections[i][4] > threshold:
                    suppressed.append(detections[i])
                    for j in range(i+1, len(detections)):
                        if iou(detections[i][:4], detections[j][:4]) > threshold:
                            break
                    else:
                        suppressed.append(detections[i])
            return suppressed
        return []

    def iou(box_a, box_b):
        x1, y1, w1, h1 = box_a
        x2, y2, w2, h2 = box_b
        intersection_area = max(0, min(x1+w1, x2+w2)-max(x1,x2)) * max(0, min(y1+h1, y2+h2)-max(y1,y2))
        union_area = float(w1*h1 + w2*h2 - intersection_area)
        return intersection_area / union_area

    # First pass: NMS for Haar cascade detections
    haarsuppressed = nms_boxes([(x, y, w, h, 0.8) for x, y, w, h, _ in combined_results], 0.3)

    # Second pass: NMS for deep learning detections
    dnnsuppressed = nms_boxes([(x, y, w, h, confidence) for x, y, w, h, confidence in combined_results], 0.3)

    # Combine results, keeping only detections that appear in both methods
    final_detections = []
    for x, y, w, h, _ in haarsuppressed:
        if any(x <= det_x < x+w and y <= det_y < y+h and x+w > det_x and y+h > det_y for det_x, det_y, _, _, _ in dnnsuppressed):
            final_detections.append((x, y, w, h, 0.8))  # Using Haar cascade confidence as default

    return final_detections

def detect_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_classes[gender_preds[0].argmax()]
    return gender

def get_optimal_font_scale(text, width, height):
    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale/10, 2)
        new_width = textSize[0][0]
        new_height = textSize[0][1]
        if (new_width <= width*0.95 and new_height <= height*0.95):
            return scale/10
    return 1

def process_images(image_paths):
    for image_path in image_paths:
        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image {image_path}")
                continue
            
            faces = detect_faces(image)

            print(f"\nDetections for image {image_paths.index(image_path)+1}:")
            for face in faces:
                x, y, w, h, confidence = face
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Extract face region
                face_region = image[y:y+h, x:x+w]
                
                # Detect gender
                gender = detect_gender(face_region)

                # Calculate optimal font scale
                text_width = image.shape[1] * 0.8
                text_height = image.shape[0] * 0.05
                font_scale = get_optimal_font_scale(f"Face {image_paths.index(image_path)+1}", text_width, text_height)
                
                # Display text
                cv2.putText(image, f"{gender}", (x, y+h+30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), 2)
            
            cv2.imshow("Faces", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            print("\nFinal detections:")
            for face in faces:
                x, y, w, h, confidence = face
                print(f"Face {image_paths.index(image_path)+1}:")
                print(f"  Position: ({x}, {y}) - ({x+w}, {y+h})")
                print(f"  Size: Width={w}, Height={h}")
                print(f"  Confidence: {confidence:.2f}")
                print(f"  Gender: {gender}")
                print("---")

        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")

# Test the improved detection on all images
process_images(['img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg', 'img7.jpg'])


Detections for image 1:

Final detections:
Face 1:
  Position: (22, 11) - (73, 62)
  Size: Width=51, Height=51
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (304, 11) - (360, 67)
  Size: Width=56, Height=56
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (304, 11) - (360, 67)
  Size: Width=56, Height=56
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (121, 100) - (172, 151)
  Size: Width=51, Height=51
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (121, 100) - (172, 151)
  Size: Width=51, Height=51
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (114, 197) - (173, 256)
  Size: Width=59, Height=59
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (114, 197) - (173, 256)
  Size: Width=59, Height=59
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (400, 197) - (454, 251)
  Size: Width=54, Height=54
  Confidence: 0.80
  Gender: Female
---
Face 1:
  Position: (400, 197) - (454, 251)
  Size: Width=54, Height